# Eat Safe, Love

## Notebook Set Up

Data Source: [UK Food Standards Agency](https://www.food.gov.uk/) (2022). UK food hygiene rating data API https://ratings.food.gov.uk/open-data/en-GB. Contains public sector information licensed under the [Open Government Licence v3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)<br />
Accessed Sept 9, 2022 with the establishment settings as follows: longitude=51.5072, latitude=-0.1276, maxdistancelimit=4567, pagesize=10000, sortoptionkey=distance, pagenumber=(1,2,3).

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [9]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
print("Number of documents in result: ", establishments.count_documents(query))
print("First result:")
results = establishments.find(query)
pprint(results[0])

Number of documents in result:  0
First result:


IndexError: no such item for Cursor instance

In [8]:
# Convert the result to a Pandas DataFrame
hygiene_20 = establishments.find(query)
hygiene_20_df = pd.DataFrame(hygiene_20)
print("Documents in DataFrame: ", len(hygiene_20_df))
hygiene_20_df.head(10)

Documents in DataFrame:  0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4}}
print("Number of documents in result: ", establishments.count_documents(query))
print("First result:")
results = establishments.find(query)
pprint(results[0])

Number of documents in result:  0
First result:


IndexError: no such item for Cursor instance

In [9]:
# Convert the result to a Pandas DataFrame
london_4plus = establishments.find(query)
london_4plus_df = pd.DataFrame(london_4plus)
print("Documents in DataFrame: ", len(london_4plus_df))
london_4plus_df.head(10)

Documents in DataFrame:  0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Create the pipeline: match, then sort, then limit to 5
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'geocode.latitude': {'$gte':latitude-degree_search, '$lte':latitude+degree_search},
         'geocode.longitude': {'$gte': longitude-degree_search, '$lte': longitude+degree_search},
         'RatingValue': 5}
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

[]


In [11]:
# Convert result to Pandas DataFrame
near_penang_flavours = establishments.find(query).sort(sort).limit(limit)
near_penang_flavours_df = pd.DataFrame(near_penang_flavours)
near_penang_flavours_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by LocalAuthorityName
# 3. Sorts the matches from highest to lowest
# Mongosh version:
# db.establishments.aggregate( [ { $match: {'scores.Hygiene': 0}}, { $group: { _id: "$LocalAuthorityName", count: { $sum: 1 }}}])
pipeline = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}},
    {'$sort': { 'count': -1 }}
]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents in result: ", len(results))

# Print the first 10 results
pprint(results[0:10])


Number of documents in result:  0
[]


In [13]:
# Convert mongo result to Pandas DataFrame
result_df = pd.DataFrame(results)

print("Rows in DataFrame: ", len(result_df))
result_df.head(10)

Rows in DataFrame:  0


""
